In [1]:
import pandas as pd
import numpy as np
import wfdb
import os
import json

In [2]:
# set working directory first
wd = 'E:/ECG Analysis'
destination = 'E:/ECG Analysis/Dataset'
patient_files = [file for file in os.listdir("E:/ECG Analysis/PTDBD ECG/") if 'patient' in file]

In [3]:
for participant in patient_files:
    path = wd + "\\PTDBD ECG\\" + participant
    filename_all = [i for i in os.listdir(path) if ".dat" in i]
    os.mkdir(destination+'/'+participant)
    patient_destination=destination+'/'+participant
    #creating csv for the individual ECG signal data recorded
    for filename in filename_all:
        file = path + "\\" + filename
        # get signal
        data = dict()
        for i in range(15):
            signal_name = wfdb.rdsamp(record_name=file[:-4])[1]['sig_name'][i]
            signal = wfdb.rdsamp(record_name=file[:-4])[0][:, i]
            data[str(signal_name)]=signal
        df=pd.DataFrame(data)
        #saving ECG signal CSV
        df.to_csv(patient_destination+'/'+filename[:-4]+'.csv')
    #creating metadata json
    meta_data = dict()
    meta_data["Participant"] = participant
    meta_data["Sampling_Rate"] = 1000
    meta_data["Database"] = "PTB-Diagnostic-ECG-Database-1.0.0"
    try:
        meta_data["Sex"] = wfdb.rdsamp(file[:-4])[1]['comments'][1][5:]
        meta_data["Age"] = wfdb.rdsamp(file[:-4])[1]['comments'][0][5:]
    except:
        meta_data["Sex"] = "Unknown"
        meta_data["Age"] = "Unknown"
    #opening header file
    header_file=[i for i in os.listdir(path) if ".hea" in i][0]
    with open(path+"/"+header_file) as f:
        d=f.readlines()
    meta_data["Diagnosis"] = [i[24:][:-1] for i in d if "Reason for admission" in i][0]
    # saving metadata
    out_file = open(os.path.join(patient_destination,"metadata.json"), "w")  
    json.dump(meta_data, out_file, indent = 6)  
    out_file.close()